# Classificações, Árvore de decisão


Projeto relaciona tabela com milhas dos carros, ano do modelo e preço e se foi vendido. O objetivo é saber se o carro vai ser vendido dependendo das condições que ele recebeu.

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"
dados = pd.read_csv(uri)
dados.head()

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


Ajustando os dados (renomeando coluna e passando 0 e 1 para a coluna de vendido)

In [2]:
a_renomear = {
    'mileage_per_year' : 'milhas_por_ano',
    'model_year' : 'ano_do_modelo',
    'price' : 'preco',
    'sold':'vendido'
}

dados = dados.rename(columns=a_renomear)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [3]:
a_trocar = {
    'no' : 0,
    'yes' : 1
}
dados.vendido = dados.vendido.map(a_trocar)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


Por opção é melhor trabalhar com a idade do carro, pois o valor é mais relacionado a idade

In [4]:
from datetime import datetime

ano_atual = datetime.today().year
dados['idade_do_modelo'] = ano_atual - dados.ano_do_modelo
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,19
1,1,7843,1998,40557.96,1,21
2,2,7109,2006,89627.50,0,13
3,3,26823,2015,95276.14,0,4
4,4,7935,2014,117384.68,1,5


Convertendo para KM (Nosso padrão brasileiro)

In [5]:
dados['km_por_ano'] = dados.milhas_por_ano * 1.60934
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,19,35085.22134
1,1,7843,1998,40557.96,1,21,12622.05362
2,2,7109,2006,89627.50,0,13,11440.79806
3,3,26823,2015,95276.14,0,4,43167.32682
4,4,7935,2014,117384.68,1,5,12770.11290


Removendo colunas do modelo, com o método <code>.drop</code>. Por padrão o drop remove linhas e não colunas, então passamos o eixo para remoção

In [6]:
dados = dados.drop(columns = ["Unnamed: 0", "milhas_por_ano", "ano_do_modelo"], axis=1)

dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,19,35085.22134
1,40557.96,1,21,12622.05362
2,89627.50,0,13,11440.79806
3,95276.14,0,4,43167.32682
4,117384.68,1,5,12770.11290


Agora escolheremos os dados que serão treinados. X -> O que queremos, Y-> Resultado que queremos obter

In [7]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

import numpy as np
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

SEED = 20
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, 
                                                        test_size = 0.25,
                                                        stratify = y)

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)

previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100

print ("%2.f%%" % acuracia)

58%


c:\users\lucas.azevedo\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Para se ter uma base se a análise é boa ou não, precisamos de uma base de classificação (ex: Chutar todos os valores com 1). O SKLearn, já possui este estimador <code>from sklear.dummy import DummyClassifier</code> e com ele, podemos fazser como todos os outros dados e ver o resultado para comparar.<br>
Stretified pega os valores proporcionais.<br>
Mostfrequent pega os valores mais frequentes.

In [8]:
from sklearn.dummy import DummyClassifier

dummy_stretified = DummyClassifier()

dummy_stretified.fit(treino_x, treino_y)
previsoes = dummy_stretified.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100

print ("%2.f%%" % acuracia)

52%


A acuracia já pode ser classificada direto utilizando <code>.score</code>

In [9]:
from sklearn.dummy import DummyClassifier

dummy_stretified = DummyClassifier()

dummy_stretified.fit(treino_x, treino_y)
acuracia = dummy_stretified.score(teste_y, teste_y) * 100

print ("%2.f%%" % acuracia)

51%


#### Agora utilizando o SVC

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

SEED = 20
np.random.seed(SEED)

raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, 
                                                        random_state = SEED, 
                                                        test_size = 0.25,
                                                        stratify = y)
scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = SVC()
modelo.fit(treino_x, treino_y)

previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100

print ("%2.f%%" % acuracia)

76%


## Árvore de decisão (Decision Tree)

Os classificadores utilizados até agora, não explicam o pq das decisões tomadas, e ficamos dependentes deles. Então passaremos a utilizar a árvore de decisão que nos mostrará passo a passo e entenderemos nossos dados.